In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train/train.csv')

In [3]:
df.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df=df.dropna()

In [5]:
X=df.drop('label',axis=1)

In [6]:
y=df['label']

In [7]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:

messages=X.copy()
messages.reset_index(inplace=True)

In [9]:
#from nltk.stem.porter import PorterStemmer
#ps = PorterStemmer()
lr = WordNetLemmatizer()
corpus = []
for i in range(0, len(messages)):
    print("Status: %s / %s" %(i, len(messages)), end="\r")
    review = re.sub('[^a-zA-Z]', ' ', messages['text'][i])
    review = review.lower()
    review = review.split()
    
    review = [lr.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [10]:
corpus[0]

'house dem aide even see comey letter jason chaffetz tweeted darrell lucus october subscribe jason chaffetz stump american fork utah image courtesy michael jolley available creative common license apology keith olbermann doubt worst person world week fbi director james comey according house democratic aide look like also know second worst person well turn comey sent infamous letter announcing fbi looking email may related hillary clinton email server ranking democrat relevant committee hear comey found via tweet one republican committee chairman know comey notified republican chairman democratic ranking member house intelligence judiciary oversight committee agency reviewing email recently discovered order see contained classified information long letter went oversight committee chairman jason chaffetz set political world ablaze tweet fbi dir informed fbi learned existence email appear pertinent investigation case reopened jason chaffetz jasoninthehouse october course know case comey a

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v=TfidfVectorizer(max_features=500,ngram_range=(1,3))
X=tfidf_v.fit_transform(corpus).toarray()

In [51]:
X.shape

(18285, 500)

In [52]:
X=X.reshape(18285,500,-1)

In [53]:
X.shape

(18285, 500, 1)

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [57]:
model = Sequential()
model.add(LSTM(units=64, input_shape =(500,1),activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='relu'))
model.add(tf.keras.layers.Activation('sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dropout_8 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 16)                1040      
_________________________________________________________________
dropout_9 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 17        
_________________________________________________________________
activation (Activation)      (None, 1)                 0         
Total params: 17,953
Trainable params: 17,953
Non-trainable params: 0
__________________________________________________

In [58]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [33]:
len(X_train)

12250

In [34]:
len(X_test)

6035

In [59]:
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=256)

Epoch 1/10
39/39 [==============================] - 60s 2s/step - loss: 0.6932 - accuracy: 0.5635 - val_loss: 0.6931 - val_accuracy: 0.5629
Epoch 2/10
39/39 [==============================] - 54s 1s/step - loss: 0.6931 - accuracy: 0.5677 - val_loss: 0.6931 - val_accuracy: 0.5629
Epoch 3/10
39/39 [==============================] - 55s 1s/step - loss: 0.6931 - accuracy: 0.5677 - val_loss: 0.6931 - val_accuracy: 0.5629
Epoch 4/10
39/39 [==============================] - 54s 1s/step - loss: 0.6931 - accuracy: 0.5677 - val_loss: 0.6931 - val_accuracy: 0.5629
Epoch 5/10
39/39 [==============================] - 54s 1s/step - loss: 0.6931 - accuracy: 0.5677 - val_loss: 0.6931 - val_accuracy: 0.5629
Epoch 6/10
39/39 [==============================] - 59s 2s/step - loss: 0.6931 - accuracy: 0.5677 - val_loss: 0.6931 - val_accuracy: 0.5629
Epoch 7/10
39/39 [==============================] - 63s 2s/step - loss: 0.6931 - accuracy: 0.5677 - val_loss: 0.6931 - val_accuracy: 0.5629
Epoch 8/10
39/39 [==

In [68]:
import numpy as np
y_pred1=model.predict_classes(X_test)

In [69]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.5665285832642917

In [62]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred1)

array([[3419,    0],
       [2616,    0]], dtype=int64)

In [63]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.57      1.00      0.72      3419
           1       0.00      0.00      0.00      2616

    accuracy                           0.57      6035
   macro avg       0.28      0.50      0.36      6035
weighted avg       0.32      0.57      0.41      6035



C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
